<a href="https://colab.research.google.com/github/roxoho/tennsorflow_learning/blob/main/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
from IPython.display import clear_output

import tensorflow as tf
import pandas as pd

In [2]:
column_names = ['SepalLength', 'SepalWidth', 'PetalLength','PetalWidth', 'Species']
species = ['Setosa', 'Versicolor', 'Virginica']

In [3]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"
)
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv"
)

train = pd.read_csv(train_path, names=column_names, header=0)
test = pd.read_csv(test_path, names=column_names, header=0)

573/573 [==============================] - 0s 0us/step


In [4]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [5]:
train.shape

(120, 5)

In [6]:
train_y = train.pop('Species')
test_y = test.pop('Species')

In [7]:
def input_function(features,labels,training=True,batch_size=256):
  ds = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  if training:
    ds = ds.shuffle(1000).repeat()
  return ds.batch(batch_size)

In [8]:
my_feature_columns=[]
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [9]:
classifier = tf.estimator.DNNClassifier(
    feature_columns = my_feature_columns,
    hidden_units = [30,10],
    n_classes=3
)

Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.


In [13]:
classifier.train(
    input_fn=lambda: input_function(train, train_y, training = True),
    steps=5000
)
clear_output()

In [14]:
result = classifier.evaluate(
    input_fn=lambda: input_function(test, test_y, training = False),
    steps=5000
)
clear_output()
print('\nTest Set Accuracy: {accuracy:0.3f}\n'.format(**result))


Test Set Accuracy: 0.933



In [22]:
def input_fn(features,batch_size=256):
  ds = tf.data.Dataset.from_tensor_slices((dict(features))).batch(batch_size)
  return ds

features = ['SepalLength', 'SepalWidth', 'PetalLength','PetalWidth']
predict = {}

print("Please Type numeric values as prompted")
for feature in features:
  valid = True
  while valid:
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature]=[float(val)]

predictions = classifier.predict(input_fn=lambda:input_fn(predict))
for pred_dict in predictions:
  class_id=pred_dict['class_ids'][0]
  probability=pred_dict['probabilities'][class_id]

  print('Prediction is "{}" ({:.1f}%)'.format(
      species[class_id],100*probability
  ))

Please Type numeric values as prompted
SepalLength: 0.5
SepalWidth: 2.0
PetalLength: 1.0
PetalWidth: 1.5
Prediction is "Versicolor" (44.8%)
